In [0]:
# # Load bronze data for testing
# row_number = 1
# df_all_bronze = spark.table("data.bronze.bronze_A75").orderBy("filename")
# df_bronze_row = df_all_bronze.limit(row_number).tail(1)
# df_bronze_single = spark.createDataFrame(df_bronze_row, df_all_bronze.schema)

# df_bronze_single.display()

In [0]:
# from pyspark.sql.functions import col, explode, to_timestamp, regexp_extract, from_xml, schema_of_xml, array
# from pyspark.sql import DataFrame

# def extract_timeseries(df_input: DataFrame) -> DataFrame:
#     """
#     Extract each TimeSeries from XML as a separate row.
    
#     Args:
#         df_input: DataFrame with columns: filename, current_timestamp, xml_string
        
#     Returns:
#         DataFrame with one row per TimeSeries, including the TimeSeries struct and psrType
#     """
#     # Extract metadata
#     df_base = df_input.select(
#         col("filename"),
#         col("current_timestamp").alias("ingestion_timestamp"),
#         col("xml_string")
#     )
    
#     # Infer schema from a sample XML
#     sample_xml = df_base.select("xml_string").first()[0]
#     xml_schema = schema_of_xml(sample_xml)
    
#     # Parse XML into struct
#     df_parsed_xml = df_base.withColumn("parsed", from_xml(col("xml_string"), xml_schema))
    
#     # Check if TimeSeries is an array or struct
#     timeseries_field = df_parsed_xml.select("parsed.TimeSeries").schema.fields[0]
#     is_timeseries_array = str(timeseries_field.dataType).startswith("ArrayType")
    
#     # Extract and explode TimeSeries
#     if is_timeseries_array:
#         # TimeSeries is already an array
#         df_timeseries = df_parsed_xml.select(
#             col("filename"),
#             col("ingestion_timestamp"),
#             explode(col("parsed.TimeSeries")).alias("timeseries")
#         )
#     else:
#         # TimeSeries is a struct - wrap in array first
#         df_timeseries = df_parsed_xml.select(
#             col("filename"),
#             col("ingestion_timestamp"),
#             explode(array(col("parsed.TimeSeries"))).alias("timeseries")
#         )
    
#     # Extract psrType from the timeseries struct
#     df_timeseries_with_psrtype = df_timeseries.select(
#         col("filename"),
#         col("ingestion_timestamp"),
#         col("timeseries.MktPSRType.psrType").alias("psrType"),
#         col("timeseries")
#     )
    
#     return df_timeseries_with_psrtype

# # Test with the same row
# row_number = 1
# df_all_bronze = spark.table("data.bronze.bronze_A75").orderBy("filename")
# df_bronze_row = df_all_bronze.limit(row_number).tail(1)
# df_bronze_single = spark.createDataFrame(df_bronze_row, df_all_bronze.schema)

# # Extract TimeSeries
# df_timeseries = extract_timeseries(df_bronze_single)

# df_timeseries.display()

In [0]:
# from pyspark.sql.functions import col, explode, array, to_timestamp
# from pyspark.sql import DataFrame

# def extract_periods(df_timeseries: DataFrame) -> DataFrame:
#     """
#     Extract each Period from TimeSeries as a separate row.
    
#     Args:
#         df_timeseries: DataFrame with columns: filename, ingestion_timestamp, psrType, timeseries
        
#     Returns:
#         DataFrame with one row per Period, including the Period struct and interval start/end
#     """
#     # Check if Period is an array or struct by inspecting the schema
#     period_field = df_timeseries.select("timeseries.Period").schema.fields[0]
#     is_period_array = str(period_field.dataType).startswith("ArrayType")
    
#     # Extract and explode Period
#     if is_period_array:
#         # Period is already an array
#         df_periods = df_timeseries.select(
#             col("filename"),
#             col("ingestion_timestamp"),
#             col("psrType"),
#             explode(col("timeseries.Period")).alias("period")
#         )
#     else:
#         # Period is a struct - wrap in array first
#         df_periods = df_timeseries.select(
#             col("filename"),
#             col("ingestion_timestamp"),
#             col("psrType"),
#             explode(array(col("timeseries.Period"))).alias("period")
#         )
    
#     # Extract interval_start and interval_end from the period
#     df_periods_with_intervals = df_periods.select(
#         col("filename"),
#         col("ingestion_timestamp"),
#         col("psrType"),
#         to_timestamp(col("period.timeInterval.start"), "yyyy-MM-dd'T'HH:mm'Z'").alias("interval_start"),
#         to_timestamp(col("period.timeInterval.end"), "yyyy-MM-dd'T'HH:mm'Z'").alias("interval_end"),
#         col("period")
#     )
    
#     return df_periods_with_intervals

# # Extract Periods from the timeseries
# df_periods = extract_periods(df_timeseries)

# df_periods.display()

In [0]:
# from pyspark.sql.functions import col, explode, expr
# from pyspark.sql import DataFrame

# def explode_points(df_periods: DataFrame) -> DataFrame:
#     """
#     Explode Points from each Period and calculate position_timestamp.
    
#     Args:
#         df_periods: DataFrame with columns: filename, ingestion_timestamp, psrType, interval_start, interval_end, period
        
#     Returns:
#         DataFrame with one row per Point, including position_timestamp
#     """
#     # Explode the Point array to get one row per Point
#     df_points_exploded = df_periods.select(
#         col("filename"),
#         col("ingestion_timestamp"),
#         col("psrType"),
#         col("interval_start"),
#         col("interval_end"),
#         explode(col("period.Point")).alias("point")
#     )
    
#     # Extract position and quantity from each Point
#     df_result = df_points_exploded.select(
#         col("filename"),
#         col("ingestion_timestamp"),
#         col("psrType"),
#         col("interval_start"),
#         col("interval_end"),
#         col("point.position").cast("int").alias("position"),
#         col("point.quantity").cast("double").alias("quantity")
#     )
    
#     # Calculate position_timestamp: interval_start + (position - 1) * 15 minutes
#     df_final = df_result.withColumn(
#         "position_timestamp",
#         expr("interval_start + make_interval(0, 0, 0, 0, 0, (position - 1) * 15)")
#     )
    
#     return df_final

# # Explode points and calculate timestamps
# df_final = explode_points(df_periods)

# df_final.display()

In [0]:
# from pyspark.sql.functions import col, first
# from pyspark.sql import DataFrame

# def pivot_psrtype(df_final: DataFrame) -> DataFrame:
#     """
#     Pivot the dataframe so each psrType becomes a column.
    
#     Args:
#         df_final: DataFrame with columns: filename, ingestion_timestamp, psrType, interval_start, interval_end, position, quantity, position_timestamp
        
#     Returns:
#         DataFrame pivoted on psrType with position_timestamp as the key
#     """
#     # Group by position_timestamp and metadata columns, then pivot on psrType
#     df_pivoted = df_final.groupBy(
#         "position_timestamp",
#         "filename",
#         "ingestion_timestamp",
#         "interval_start",
#         "interval_end"
#     ).pivot("psrType").agg(first("quantity"))
    
#     return df_pivoted

# # Pivot the dataframe
# df_pivoted = pivot_psrtype(df_final)

# df_pivoted.display()

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, explode, expr, to_timestamp, regexp_extract, from_xml, schema_of_xml, array, first
from pyspark.sql import DataFrame

# Define modular parsing functions
def extract_timeseries(df_input: DataFrame) -> DataFrame:
    """
    Extract each TimeSeries from XML as a separate row.
    """
    df_base = df_input.select(
        col("filename"),
        col("current_timestamp").alias("ingestion_timestamp"),
        to_timestamp(regexp_extract(col("filename"), r"(\d{12})\.xml$", 1), "yyyyMMddHHmm").alias("file_timestamp"),
        col("xml_string")
    )
    
    sample_xml = df_base.select("xml_string").first()[0]
    xml_schema = schema_of_xml(sample_xml)
    df_parsed_xml = df_base.withColumn("parsed", from_xml(col("xml_string"), xml_schema))
    
    timeseries_field = df_parsed_xml.select("parsed.TimeSeries").schema.fields[0]
    is_timeseries_array = str(timeseries_field.dataType).startswith("ArrayType")
    
    if is_timeseries_array:
        df_timeseries = df_parsed_xml.select(
            col("filename"),
            col("ingestion_timestamp"),
            col("file_timestamp"),
            explode(col("parsed.TimeSeries")).alias("timeseries")
        )
    else:
        df_timeseries = df_parsed_xml.select(
            col("filename"),
            col("ingestion_timestamp"),
            col("file_timestamp"),
            explode(array(col("parsed.TimeSeries"))).alias("timeseries")
        )
    
    df_timeseries_with_psrtype = df_timeseries.select(
        col("filename"),
        col("ingestion_timestamp"),
        col("file_timestamp"),
        col("timeseries.MktPSRType.psrType").alias("psrType"),
        col("timeseries")
    )
    
    return df_timeseries_with_psrtype

def extract_periods(df_timeseries: DataFrame) -> DataFrame:
    """
    Extract each Period from TimeSeries as a separate row.
    """
    period_field = df_timeseries.select("timeseries.Period").schema.fields[0]
    is_period_array = str(period_field.dataType).startswith("ArrayType")
    
    if is_period_array:
        df_periods = df_timeseries.select(
            col("filename"),
            col("ingestion_timestamp"),
            col("file_timestamp"),
            col("psrType"),
            explode(col("timeseries.Period")).alias("period")
        )
    else:
        df_periods = df_timeseries.select(
            col("filename"),
            col("ingestion_timestamp"),
            col("file_timestamp"),
            col("psrType"),
            explode(array(col("timeseries.Period"))).alias("period")
        )
    
    df_periods_with_intervals = df_periods.select(
        col("filename"),
        col("ingestion_timestamp"),
        col("file_timestamp"),
        col("psrType"),
        to_timestamp(col("period.timeInterval.start"), "yyyy-MM-dd'T'HH:mm'Z'").alias("interval_start"),
        to_timestamp(col("period.timeInterval.end"), "yyyy-MM-dd'T'HH:mm'Z'").alias("interval_end"),
        col("period")
    )
    
    return df_periods_with_intervals

def explode_points(df_periods: DataFrame) -> DataFrame:
    """
    Explode Points from each Period and calculate position_timestamp.
    """
    df_points_exploded = df_periods.select(
        col("filename"),
        col("ingestion_timestamp"),
        col("file_timestamp"),
        col("psrType"),
        col("interval_start"),
        col("interval_end"),
        explode(col("period.Point")).alias("point")
    )
    
    df_result = df_points_exploded.select(
        col("filename"),
        col("ingestion_timestamp"),
        col("file_timestamp"),
        col("psrType"),
        col("interval_start"),
        col("interval_end"),
        col("point.position").cast("int").alias("position"),
        col("point.quantity").cast("double").alias("quantity")
    )
    
    df_final = df_result.withColumn(
        "position_timestamp",
        expr("interval_start + make_interval(0, 0, 0, 0, 0, (position - 1) * 15)")
    )
    
    return df_final

def pivot_psrtype(df_final: DataFrame) -> DataFrame:
    """
    Pivot the dataframe so each psrType becomes a column.
    """
    df_pivoted = df_final.groupBy(
        "position_timestamp",
        "filename",
        "ingestion_timestamp",
        "file_timestamp",
        "interval_start",
        "interval_end"
    ).pivot("psrType").agg(first("quantity"))
    
    df_pivoted = df_pivoted.orderBy("position_timestamp")
    
    return df_pivoted

# Define parse_xml_row function using modular functions
def parse_xml_row(df_input: DataFrame) -> DataFrame:
    """
    Parse XML data using modular functions and pivot on psrType.
    """
    df_timeseries = extract_timeseries(df_input)
    df_periods = extract_periods(df_timeseries)
    df_points = explode_points(df_periods)
    df_pivoted = pivot_psrtype(df_points)
    
    return df_pivoted

# Define the process_batch function for foreachBatch
def process_batch(batch_df, batch_id):
    """
    Process a micro-batch of bronze data and MERGE into silver table.
    """
    print(f"\n{'='*60}")
    print(f"Processing batch {batch_id}")
    print(f"{'='*60}")
    
    row_count = batch_df.count()
    print(f"Batch contains {row_count} bronze rows")
    
    if row_count == 0:
        print("Empty batch, skipping...")
        return
    
    
    silver_table = DeltaTable.forName(spark, "data.silver.silver_A75")
    
    processed_count = 0
    for row in batch_df.toLocalIterator():
        processed_count += 1
        filename = row['filename']
        print(f"\n  [{processed_count}/{row_count}] Processing: {filename}")
        
        df_single_row = spark.createDataFrame([row], batch_df.schema)
        df_parsed = parse_xml_row(df_single_row)
        
        # Build merge condition and update set dynamically based on columns
        merge_condition = "target.position_timestamp = source.position_timestamp"
        
        # Get all columns for update
        update_set = {col_name: f"source.{col_name}" for col_name in df_parsed.columns}
        
        silver_table.alias("target").merge(
            df_parsed.alias("source"),
            merge_condition
        ).whenMatchedUpdate(
            condition="source.file_timestamp > target.file_timestamp",
            set=update_set
        ).whenNotMatchedInsertAll(
        ).execute()
        
        print(f"    ✓ MERGE completed")
    
    print(f"\n✓ Batch {batch_id} completed: {processed_count} files processed")

print("✓ parse_xml_row and process_batch functions defined")
print("Ready to use with foreachBatch in streaming")

In [0]:
# Create silver table for A75 data with FLOAT columns
spark.sql("""
CREATE TABLE if not exists data.silver.silver_A75 (
    position_timestamp TIMESTAMP,
    filename STRING,
    ingestion_timestamp TIMESTAMP,
    file_timestamp TIMESTAMP,
    interval_start TIMESTAMP,
    interval_end TIMESTAMP,
    B01 FLOAT,
    B04 FLOAT,
    B05 FLOAT,
    B11 FLOAT,
    B14 FLOAT,
    B16 FLOAT,
    B17 FLOAT,
    B18 FLOAT,
    B19 FLOAT,
    B20 FLOAT
)
USING DELTA
COMMENT 'Silver table for A75 generation load data with pivoted psrType columns'
""")

print("✓ Silver table data.silver.silver_A75 created successfully with FLOAT columns")
print("\nTable schema:")
spark.table("data.silver.silver_A75").printSchema()

In [0]:
# Setup streaming from bronze to silver
print("Setting up structured streaming...")

# Define checkpoint location
checkpoint_location = "/Volumes/source/source_schema/source_volume/checkpoints/A75_silver"

# Read bronze table as a stream
df_bronze_stream = spark.readStream.table("data.bronze.bronze_A75")

print(f"✓ Streaming source configured")
print(f"Checkpoint location: {checkpoint_location}")

# Start the streaming query with foreachBatch
print("\nStarting streaming query...")
print("Mode: Batch (trigger availableNow - processes all available data once)\n")

query = df_bronze_stream.writeStream \
    .foreachBatch(process_batch) \
    .option("checkpointLocation", checkpoint_location) \
    .trigger(availableNow=True) \
    .start()

# Wait for the stream to finish processing
query.awaitTermination()

print("\n" + "="*60)
print("✓ Streaming query completed successfully!")
print("="*60)
print("\nAll unprocessed bronze rows have been processed to silver.")
print("Next run will only process new rows added after this checkpoint.")

In [0]:
# checkpoint_location = "/Volumes/source/source_schema/source_volume/checkpoints/A75_silver"
# dbutils.fs.rm(checkpoint_location, recurse=True)


In [0]:
# %sql
# drop table data.silver.silver_A75